In [1]:
%pip install python-dotenv google-generativeai llama-index tiktoken

In [2]:
import os
import tiktoken

from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

from llama_index.core import VectorStoreIndex, Document
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.indices.postprocessor import SimilarityPostprocessor
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response.pprint_utils import pprint_response

# Load data from a text file
def load_data_from_txt(file_path, encodings=('utf-8', 'cp1252')):
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as file:
                content = file.read()
            return [Document(text=content)]
        except UnicodeDecodeError:
            continue
    raise Exception("Unable to decode the file using specified encodings")

In [3]:
# Function to count tokens
def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [4]:
def num_tokens_from_response(response, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(response.response))
    return num_tokens

In [5]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [10]:
# Assuming 'data.txt' is your file
file_path = "cleaned_text_output.txt"
documents = load_data_from_txt(file_path)

In [13]:

# Create the index
index = VectorStoreIndex.from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

ValueError: 
******
Could not load OpenAI embedding model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

Consider using embed_model='local'.
Visit our documentation for more embedding options: https://docs.llamaindex.ai/en/stable/module_guides/models/embeddings.html#modules
******

In [9]:

# Set up the retriever and postprocessor
retriever = VectorIndexRetriever(index=index, similarity_top_k=4)
postprocessor = SimilarityPostprocessor(similarity_cutoff=0.75)

In [10]:
# Create the query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[postprocessor],
)

In [42]:
# Define the cost calculation function
def openai_api_calculate_cost(usage, model="gpt-3.5-turbo-16k"):
    pricing = {
        'gpt-3.5-turbo-4k': {
            'prompt': 0.0015,
            'completion': 0.002,
        },
        'gpt-3.5-turbo-16k': {
            'prompt': 0.003,
            'completion': 0.004,
        },
        'gpt-4-8k': {
            'prompt': 0.03,
            'completion': 0.06,
        },
        'gpt-4-32k': {
            'prompt': 0.06,
            'completion': 0.12,
        },
        'text-embedding-ada-002-v2': {
            'prompt': 0.0001,
            'completion': 0.0001,
        }
    }

    try:
        model_pricing = pricing[model]
    except KeyError:
        raise ValueError("Invalid model specified")

    prompt_cost = usage['prompt_tokens'] * model_pricing['prompt'] / 1000
    completion_cost = usage['completion_tokens'] * model_pricing['completion'] / 1000

    total_cost = prompt_cost + completion_cost
    print(f"\nTokens used:  {usage['prompt_tokens']:,} prompt + {usage['completion_tokens']:,} completion = {usage['total_tokens']:,} tokens")
    print(f"Total cost for {model}: ${total_cost:.4f}\n")

    return total_cost

In [14]:
# Perform a query
query = "Gunghat?"
# response = query_engine.query("What is Video MER?")
# response = query_engine.query("What is TELE MER?")
# response = query_engine.query("How can we contact quicksolv?")
# response = query_engine.query("Does Video MER help me detect heartrate?")
# response = query_engine.query("Aadhar masking and IRDA?")
response = query_engine.query(query)
# response = query_engine.query("where is Xangars located in Malaysia?")

# Print the response with source
pprint_response(response, show_source=True)

Final Response: Gunghat refers to a case of medical deception that was
discussed in a blog post by Dr. Birbal.
______________________________________________________________________
Source Node 1/1
Node ID: cc1666b1-c43b-4ec3-a224-2b010d314c64
Similarity: 0.7701762671276723
Text: Birbal Blog Internet of Things (IoT) Dr. Birbal, who is a modern
day, seasoned and highly respected health claims Investigator Dr.
Birbal Latest Blog Behind the Ghunghat: A Case of Medical Deception By
Riyaz Lakhani In Dr. Birbal Posted March 28, 2023 Behind the Ghunghat:
A Case of Medical Deception 0 Read More The Hospital Conspiracy
Unveiled: ...


In [15]:
encoding_name = "cl100k_base"  # Use the appropriate encoding for your model

query_tokens = num_tokens_from_string(query, encoding_name)
response_tokens = num_tokens_from_string(response, encoding_name)

print(f"Query: {query}")
print(f"Number of tokens in query: {query_tokens}")
print(f"Number of tokens in response: {response_tokens}")
print(f"Total number of tokens: {query_tokens + response_tokens}")

TypeError: expected string or buffer

In [13]:
print(f"{num_tokens_from_messages(response)} prompt tokens counted by num_tokens_from_messages().")

TypeError: 'Response' object is not iterable

In [56]:
num_tokens = num_tokens_from_string(response)
print(f"Query: Gunghat?")
print(f"Number of tokens: {num_tokens}")

TypeError: expected string or buffer

In [45]:
print(response)

Gunghat refers to a case of medical deception that was discussed in a blog post by Dr. Birbal.


In [46]:
if hasattr(response, 'metadata'):
    usage = {
        'prompt_tokens': response.metadata['usage']['prompt_tokens'],
        'completion_tokens': response.metadata['usage']['completion_tokens'],
        'total_tokens': response.metadata['usage']['total_tokens'],
    }
else:
    print("Token usage information not available in the response.")
    usage = None

KeyError: 'usage'

In [44]:
usage = {
    'prompt_tokens': response.usage.prompt_tokens,
    'completion_tokens': response.usage.completion_tokens,
    'total_tokens': response.usage.total_tokens,
}

AttributeError: 'Response' object has no attribute 'usage'

In [40]:
if usage:
    # Calculate and print the cost
    model_used = "gpt-3.5-turbo-16k"  # Specify the model used
    total_cost = openai_api_calculate_cost(usage, model=model_used)

NameError: name 'usage' is not defined